<a href="https://colab.research.google.com/github/Rumeysakeskin/Turkish-Text2Speech-Dataset/blob/main/download_mozilla_for_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# !sudo apt-get install libsox-fmt-all
!pip install install sox
!pip install wget
!sudo apt-get update && sudo apt-get install sox libsox-fmt-all

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetc

In [3]:
import sys
import argparse
import csv
import json
import logging
import multiprocessing
import os
import tarfile
from multiprocessing.pool import ThreadPool
from pathlib import Path
from typing import List

import sox
from sox import Transformer
import wget
from tqdm import tqdm

In [5]:
parser = argparse.ArgumentParser(description='Downloads and processes Mozilla Common Voice dataset.')
parser.add_argument("--data_root", default='CommonVoice_dataset/', type=str, help="Directory to store the dataset.")
parser.add_argument('--manifest_dir', default='./', type=str, help='Output directory for manifests')
parser.add_argument("--num_workers", default=multiprocessing.cpu_count(), type=int, help="Workers to process dataset.")
parser.add_argument('--sample_rate', default=16000, type=int, help='Sample rate')
parser.add_argument('--files_to_process', nargs='+', default=['female_data.tsv', 'male_data.tsv'],
                    type=str, help='list of *.csv file names to process')
parser.add_argument('--version', default='cv-corpus-11.0-2022-09-21',
                    type=str, help='Version of the dataset (obtainable via https://commonvoice.mozilla.org/en/datasets')
parser.add_argument('--language', default='tr',
                    type=str, help='Which language to download.(default english,'
                                   'check https://commonvoice.mozilla.org/en/datasets for more language codes')

# args = parser.parse_args()
args, unknown = parser.parse_known_args()

In [6]:
# -------- CHANGE VERSION AND LANGUAGE
version = 'cv-corpus-11.0-2022-09-21'
language = 'cv-corpus-11.0-2022-09-21-tr'

COMMON_VOICE_URL = f"https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/" \
                   "{}/{}.tar.gz".format(version, language)

In [7]:
def create_manifest(
        data: List[tuple],
        output_name: str,
        manifest_path: str):
    output_file = Path(manifest_path) / output_name
    output_file.parent.mkdir(exist_ok=True, parents=True)
    with output_file.open(mode='w', encoding="utf-8") as f:
        for wav_path, duration, text in tqdm(data, total=len(data)):
            f.write(
                json.dumps({
                    'audio_filepath': wav_path,
                    "duration": duration,
                    'text': text
                },  ensure_ascii=False) + '\n'
            )

def process_files(csv_file, data_root, num_workers):
    """ Read *.csv file description, convert mp3 to wav, process text.
        Save results to data_root.
    Args:
        csv_file: str, path to *.csv file with data description, usually start from 'cv-'
        data_root: str, path to dir to save results; wav/ dir will be created
    """
    wav_dir = os.path.join(data_root, 'wav/')
    os.makedirs(wav_dir, exist_ok=True)
    audio_clips_path = os.path.dirname(csv_file) + '/clips/'

    def process(x):
        file_path, text = x
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        text = text.lower().strip()  
        audio_path = os.path.join(audio_clips_path, file_path)
        output_wav_path = os.path.join(wav_dir, file_name + '.wav')

        tfm = Transformer()
        tfm.rate(samplerate=args.sample_rate)
        tfm.build(
            input_filepath=audio_path,
            output_filepath=output_wav_path
        )
        duration = sox.file_info.duration(output_wav_path)
        return output_wav_path, duration, text

    logging.info('Converting mp3 to wav for {}.'.format(csv_file))
    with open(csv_file, encoding="utf8") as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t')
        next(reader, None)  # skip the headers
        # data = [(row['path'], row['sentence']) for row in reader]
        data = [(row['path'], row['sentence']) for row in reader if (row['path'], row['sentence']) in row]
        with ThreadPool(num_workers) as pool:
            data = list(tqdm(pool.imap(process, data), total=len(data)))

    print(data)
    return data

In [9]:
# Install and unzip dataset
data_root = args.data_root
os.makedirs(data_root, exist_ok=True)

target_unpacked_dir = os.path.join(data_root, "CV_unpacked")

if os.path.exists(target_unpacked_dir):
    print('Find existing folder {}'.format(target_unpacked_dir))
else:
    print("Could not find Common Voice, Downloading corpus...")

    filename = wget.download(COMMON_VOICE_URL, data_root)
    target_file = os.path.join(data_root, os.path.basename(filename))

    os.makedirs(target_unpacked_dir, exist_ok=True)
    print("Unpacking corpus to {} ...".format(target_unpacked_dir))
    tar = tarfile.open(target_file, encoding="utf8")
    tar.extractall(target_unpacked_dir)
    tar.close()
    print("Done!")

Find existing folder CommonVoice_dataset/CV_unpacked


In [37]:
# Select most frequent speakers' recorded audios for male and female speakers

train_tsv = "/content/" + target_unpacked_dir + "/" + version + "/tr/train.tsv"
test_tsv = "/content/" + target_unpacked_dir + "/" + version + "/tr/test.tsv"
validated_tsv = "/content/" + target_unpacked_dir + "/" + version + "/tr/validated.tsv"

tsv_files = [train_tsv, test_tsv, validated_tsv]

from collections import Counter
id_counts = Counter()
id_genders = {}

for tsv_file in tsv_files:
  with open(tsv_file, encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    next(reader, None)
    for row in reader:
        client_id = row['client_id']
        gender = row['gender']
        id_counts[client_id] += 1
        id_genders[client_id] = gender

most_common_ids = id_counts.most_common(5)

for id, count in most_common_ids:
    print(f"Client ID: {id} | Gender: {id_genders[id]} | Count: {count}")

Client ID: ba25f19f22267b1f863cef586f17252a9067bc0ecb3d63dd7626fbce7014a16412dd64d5404d34505c0c90635adbc3cb2d830d77291a4db194e5abad91160957 | Gender: female | Count: 17252
Client ID: 6cf4dea5df1dfe5dc5d7bacdb6da105b9e48e01fd0f069dbaf0ad906a7c5d78032194a3a9384e8589b93c58a04a12f676669f4c4501e52cee652f331b72b9205 | Gender: female | Count: 17194
Client ID: c3c204ffaebfc46c0265773376f9288a372694aa79f97afe224828033c28d6d2a90919aaf769bf14105cb4033650e10275760afe250490782eae15c1d1518799 | Gender: male | Count: 6349
Client ID: 60cee2235d7ec4cdeb89d601b8c373955b303c712ec729ed0affdabda8819908f51cefa990163d2bc4ac04c93e6dac2909cca67829211df4a2a17af2507dd50a | Gender:  | Count: 6291
Client ID: ca179eb54e4e584d30587ba3c6c1d4de7d9082d649113d481d308f3e7d858e0a1df21d9591f9c613928b1e122aa7f54535d0f0e0c1380d87d8017844e6e73ea2 | Gender: male | Count: 4837


In [29]:
female_rank = input("Choose the ID number rank for the female voice dataset:")
male_rank = input("Choose the ID number rank for the male voice dataset:")
print(f"Selected female voice ID: {id_counts.most_common(int(female_rank))[int(female_rank)-1][0]}")
print(f"Selected male voice ID:   {id_counts.most_common(int(male_rank))[int(male_rank)-1][0]}")

Choose the ID number rank for the female voice dataset:1
Choose the ID number rank for the male voice dataset:3
Selected female voice ID: ba25f19f22267b1f863cef586f17252a9067bc0ecb3d63dd7626fbce7014a16412dd64d5404d34505c0c90635adbc3cb2d830d77291a4db194e5abad91160957
Selected male voice ID:   c3c204ffaebfc46c0265773376f9288a372694aa79f97afe224828033c28d6d2a90919aaf769bf14105cb4033650e10275760afe250490782eae15c1d1518799


In [11]:
# Create tsv files for new datasets
female_data = "/content/" + target_unpacked_dir + "/" + version + "/tr/female_data.tsv"
male_data = "/content/" + target_unpacked_dir + "/" + version + "/tr/male_data.tsv"

for tsv_file in tsv_files:
  with open(tsv_file, encoding="utf8") as csvfile, \
       open(female_data, mode='w', encoding='utf-8', newline='') as female_file, \
       open(male_data, mode='w', encoding='utf-8', newline='') as male_file:

      reader = csv.DictReader(csvfile, delimiter='\t')
      next(reader, None)

      fwriter = csv.DictWriter(female_file, fieldnames=reader.fieldnames, delimiter='\t')
      mwriter = csv.DictWriter(male_file, fieldnames=reader.fieldnames, delimiter='\t')
      fwriter.writeheader()
      mwriter.writeheader()
      csvfile.seek(0)
      next(csvfile, None)

      for row in reader:
        if (row['client_id'] == id_counts.most_common(int(female_rank))[int(female_rank)-1][0]):
          fwriter.writerow(row)
        if row['client_id'] == id_counts.most_common(int(male_rank))[int(male_rank)-1][0]:
          mwriter.writerow(row)

print(f"female_data.tsv and male_data.tsv files are created in /content/{target_unpacked_dir}/{version}/tr")

NameError: ignored

In [12]:
!rm -r /content/CommonVoice_dataset/female_data

In [10]:
folder_path = os.path.join("/content/"+ target_unpacked_dir, args.version + f'/{args.language}/')

for csv_file in args.files_to_process:
    data = process_files(
        csv_file=os.path.join(folder_path, csv_file),
        data_root=os.path.join(data_root, os.path.splitext(csv_file)[0]),
        num_workers=args.num_workers
    )
    logging.info('Creating manifests...')
    create_manifest(
        data=data,
        output_name=f'commonvoice_{os.path.splitext(csv_file)[0]}_manifest.jsonl',
        manifest_path=args.manifest_dir,
    )

 16%|█▋        | 9023/55351 [02:28<12:42, 60.79it/s]


OSError: ignored

In [ ]:
!zip -r /content/CommonVoice_dataset/train.zip /content/CommonVoice_dataset/train
!zip -r /content/CommonVoice_dataset/dev.zip /content/CommonVoice_dataset/dev
!zip -r /content/CommonVoice_dataset/test.zip /content/CommonVoice_dataset/test


In [ ]:
from google.colab import files
# files.download('/content/CommonVoice_dataset/dev.zip')
files.download('/content/CommonVoice_dataset/test.zip')
# files.download('/content/CommonVoice_dataset/train.zip')

In [ ]:
from google.colab import files
# files.download('/content/CommonVoice_dataset/dev.zip')
files.download('/content/CommonVoice_dataset/test.zip')